In [1]:
%run ./DFBRP_Simulator.ipynb

In [2]:
import json
import math
import numpy as np
from datetime import timedelta
from scipy import stats

In [3]:
from datetime import datetime

# start time
start_time = "2:13:57"
end_time = "11:46:38"

# convert time string to datetime
t1 = datetime.strptime(start_time, "%H:%M:%S")
print('Start time:', t1.time())

t2 = datetime.strptime(end_time, "%H:%M:%S")
print('End time:', t2.time())

# get difference
delta = t2 - t1

# time difference in seconds
print(f"Time difference is {delta.total_seconds()} seconds")

# time difference in milliseconds
ms = delta.total_seconds() * 1000
print(f"Time difference is {ms} milliseconds")

Start time: 02:13:57
End time: 11:46:38
Time difference is 34361.0 seconds
Time difference is 34361000.0 milliseconds


In [11]:
REJECTED = 0
ACCEPTED = 1
ORIGIN_INDEX = 1
DESTINATION_INDEX = 2
BUS_STOP_DATA_PATH = "./Data/bus_stops.json"
REWARD_PER_PASSENGER = 100
POSITIVE_LOAD_REWARD = 50

class GymDFBRPSimulator(DFBRPSimulator):
    def __init__(self, numberOfBuses,numberOfRequests,rendermode=None):
        super().__init__(numberOfBuses,numberOfRequests,render_mode=rendermode)
        self.numberOfBuses = numberOfBuses
        self.numberOfRequests = numberOfRequests
        self.smallestLatitude,self.smallestLongitude,self.largestLatitude,self.largestLongitude = self._findBoundariesCords()

        self.previousRewardDistance = 0
        self.previousRewardWaitingTime = 0

        if rendermode =="human":
            self.initMap()

    def _findBoundariesCords(self):
        boundaries = MAP_BOUNDARIES
        lat1 = boundaries[0]
        lon1 = boundaries[1]
        lat2 = boundaries[2]
        lon2 = boundaries[3]

        smallestLat = min(lat1,lat2)
        smallestLon = min(lon1,lon2)
        largestLat = max(lat1,lat2)
        largestLon = max(lon1,lon2)
        return smallestLat,smallestLon,largestLat,largestLon
    
    #Observation Functions
    def getRequestObservation(self):
        #Create an np array of the following format:
        # [requestLocationLatitude,requestLocationLongitude, requestDestinationLatitude,requestDestinationLongitude, requestPassengerCount]
        request = self.currentRequest
        origin = request.getOrigin()
        destination = request.getDestination()
        requestObservation = np.array([origin.getLatitude()-self.smallestLatitude,origin.getLongitude()-self.smallestLongitude,destination.getLatitude()-self.smallestLatitude,destination.getLongitude()-self.smallestLongitude,request.getPassengerAmount(),request.getId()], dtype=np.float32)
        return requestObservation 
    def getBusesLocationsObservation(self):
        busesLocations = np.zeros((self.numberOfBuses,2), dtype=np.float32)
        for i,vehicle in enumerate(self.vehicles):
            location = vehicle.getPosition()
            busesLocations[i] = [location.getLatitude()-self.smallestLatitude,location.getLongitude()-self.smallestLongitude]
        assert busesLocations.shape == (self.numberOfBuses,2)
        return busesLocations    
    def getPassengerCountsObservation(self):
        passengerCounts = np.zeros((self.numberOfBuses), dtype=np.int8)
        for i,vehicle in enumerate(self.vehicles):
            passengerCounts[i] = vehicle.getCurrentLoad()
        assert passengerCounts.shape == (self.numberOfBuses,)
        return passengerCounts  
    def getBusesRoutesObservation(self):
        busesRoutes = np.zeros((self.numberOfBuses,self.getCapacity()*2,2), dtype=np.float32)
        for i in range(self.numberOfBuses):
            busesRoutes[i] = self._getBusRoute(i)
        return busesRoutes
    
    #MISC FUNCTIONS
    def getCapacity(self):
        return self.vehicles[0].getCapacity()
    def _getBusRoute(self, index):
        route = self.vehicles[index].getListOfCords()
        npRoute = np.zeros((self.getCapacity()*2,2), dtype=np.float32)
        if route != None:
            for i in range(len(route)):
                npRoute[i] = [route[i].getLatitude()-self.smallestLatitude,route[i].getLongitude()-self.smallestLongitude]
        return npRoute
    #REWARD FUNCTION
    def getPreStepReward(self):
        distanceReward = []
        waitingTimeReward = []
        broken = self.getTimeWindowsReward()
        for vehicle in self.vehicles:
            distanceReward.append(vehicle.getRouteDistance()/1000)
            waitingTimeReward.append(vehicle.getRequestWaitingTime()/60)
        return distanceReward,broken
    def getTimeWindowsReward(self):
        rewardList = []
        for vehicle in self.vehicles:
            timeWindowsReward = 0
            if vehicle.getRouteSize() == 0:
                rewardList.append(0)
                continue
            currentTime = self.currentRequest.getTime()
            route = vehicle.getRequestOSRMJSON()
            for i in range(len(route["routes"][0]["legs"])):
                leg = route["routes"][0]["legs"][i]
                legDistance = leg["distance"]
                duration = legDistance/vehicle.speed
                currentTime = timedelta(seconds=duration) + currentTime 
                timeWindow = vehicle.routeList[i].getTimeWindow()
                timeDiff = (((timeWindow - currentTime).total_seconds())/60)
                # print("current time: "+str(currentTime) + " time window: "+str(timeWindow) + " time diff: "+str(timeDiff)+ "time diff squared: "+str(timeDiff*timeDiff))
                if timeDiff < 0:
                    timeDiff = -1*(timeDiff*timeDiff)
                # print("current time: "+str(currentTime) + " time window: "+str(timeWindow) + " time diff: "+str(timeDiff))
                timeWindowsReward += timeDiff
            rewardList.append(timeWindowsReward)
        return rewardList
    def getBusesDistancesObservation(self):
        busesDistances = np.zeros((self.numberOfBuses), dtype=np.float32)
        for i,vehicle in enumerate(self.vehicles):
            busesDistances[i] = vehicle.getRouteDistance()
        return busesDistances
    def getBusesIdleTimesObservation(self):
        busesIdleTimes = np.zeros((self.numberOfBuses), dtype=np.float32)
        for i,vehicle in enumerate(self.vehicles):
            busesIdleTimes[i] = vehicle.stats._totalTime
        return busesIdleTimes

    def getReward(self,action,prevBrokenTimeWindows):
        rewards = []
        acceptedReward = np.zeros((self.numberOfBuses), dtype=np.float32)

        if action == REJECTED:
            totalReward = self.currentRequest.getPassengerAmount()*(REWARD_PER_PASSENGER)*-1*(len(self.rejectedRequests))
            for i in range(len(self.vehicles)):
                acceptedReward[i] = totalReward

        broken = self.getTimeWindowsReward()
        brokenReward = (np.sum(broken) - np.sum(prevBrokenTimeWindows))
        for i in range(len(self.vehicles)):
            reward = 0
            reward += acceptedReward[i]
            reward += brokenReward
            rewards.append(reward)

        print(str(acceptedReward) + " + " + str(brokenReward) + " = " + str(rewards))
        return rewards

    def reset(self):
        self.vehicles = self.initVehicles(self.numberOfBuses)
        self.requests = self.initRequests(self.numberOfRequests)
        self.currentRequest = self.requests[0]
        self.currentRequestIndex = 0
        self.rejectedRequests = []
        self.handledRequest = False
        if self.render_mode == "human":
            self._updateMap()
    def getSmallestLongitude(self):
        return self.smallestLongitude
    def getSmallestLatitude(self):
        return self.smallestLatitude
    def getLargestLongitude(self):
        return self.largestLongitude
    def getLargestLatitude(self):
        return self.largestLatitude

In [7]:
# import time as ts
# d1 = GymDFBRPSimulator(10,150,rendermode="human")

# for i in range(150):
#     index = i%10
#     previousTimeWindows = d1.getTimeWindowsReward()
#     d1.acceptRequest(index,len(d1.vehicles[index].routeList),len(d1.vehicles[index].routeList)+1)
#     d1.advanceToNextRequest()
#     previousDistance,previousWaitingTime = d1.getPreStepReward()
#     print("Reward:",d1.getReward(ACCEPTED,0,0,previousTimeWindows))
#     d1.updateState()
#     print("RouteList",len(d1.vehicles[0].routeList))
#     ts.sleep(1)

Map(center=[35.9010058, 14.4507084], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

current time: 2024-05-23 08:15:44.568000 time window: 2024-05-23 08:15:00 time diff: -0.7428time diff squared: 0.55175184
current time: 2024-05-23 08:24:37.446000 time window: 2024-05-23 08:38:52.878000 time diff: 14.257200000000001time diff squared: 203.26775184000002
current time: 2024-05-23 08:15:44.568000 time window: 2024-05-23 08:15:00 time diff: -0.7428time diff squared: 0.55175184
current time: 2024-05-23 08:24:37.446000 time window: 2024-05-23 08:38:52.878000 time diff: 14.257200000000001time diff squared: 203.26775184000002
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] + 13.705448160000001 = [13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001]
Reward: [13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000001, 13.705448160000

In [8]:
# import matplotlib.pyplot as plt

In [9]:
# def plotTimeWindows(vehilcesList,requestsList,path):
#     originTimeWindow = {}
#     destinationTimeWindow = {}
#     originTimes = {}
#     destinationTimes = {}

#     originTimeWindowList = []
#     destinationTimeWindowList = []
#     originTimesList = []
#     destinationTimesList = []

#     completedInTimeOriginKeys = []
#     completedInTimeOriginTimes = []
#     exceededTimeWindowOriginKeys = []
#     exceededTimeWindowOriginTimes = []
#     completedInTimeDestinationKeys = []
#     completedInTimeDestinationTimes = []
#     exceededTimeWindowDestinationKeys = []
#     exceededTimeWindowDestinationTimes = []

#     for i in range(len(requestsList)):
#         originTimes[requestsList[i].getId()] = None
#         destinationTimes[requestsList[i].getId()] = None
#         originTimeWindow[i] = requestsList[i].getOrigin().getTimeWindow()
#         destinationTimeWindow[i] = requestsList[i].getDestination().getTimeWindow()

#     for vehicle in vehilcesList:
#         for key in vehicle.stats._originPickUpTimes:
#             originTimes[key] = vehicle.stats._originPickUpTimes[key]
#     for vehicle in vehilcesList:
#         for key in vehicle.stats._destinationPickUpTimes:
#             destinationTimes[key] = vehicle.stats._destinationPickUpTimes[key]
    
#     for key in list(originTimes):
#         if originTimes[key] == None:
#             del originTimes[key]
#     for key in list(destinationTimes):
#         if destinationTimes[key] == None:
#             del destinationTimes[key]

#     for key in originTimes:
#         if originTimeWindow[key] > originTimes[key]:
#             completedInTimeOriginKeys.append(key)
#             completedInTimeOriginTimes.append(originTimes[key])
#         else:
#             exceededTimeWindowOriginKeys.append(key)
#             exceededTimeWindowOriginTimes.append(originTimes[key])

#     for key in destinationTimes:
#         if destinationTimeWindow[key] > destinationTimes[key]:
#             completedInTimeDestinationKeys.append(key)
#             completedInTimeDestinationTimes.append(destinationTimes[key])
#         else:
#             exceededTimeWindowDestinationKeys.append(key)
#             exceededTimeWindowDestinationTimes.append(destinationTimes[key])

#     for i in range(len(requestsList)):
#         originTimeWindowList.append(originTimeWindow[i])
#         destinationTimeWindowList.append(destinationTimeWindow[i])
#         if i in originTimes:
#             originTimesList.append(originTimes[i])
#         if i in destinationTimes:
#             destinationTimesList.append(destinationTimes[i])

#     #Create a line chart of the graphs
#     fig, ax = plt.subplots()
#     ax.plot(originTimeWindowList, label="Origin Time Window", color="black", alpha=0.5)
#     ax.scatter(completedInTimeOriginKeys,completedInTimeOriginTimes, label="Completed Origin Times", color="green", s=5)
#     ax.scatter(exceededTimeWindowOriginKeys,exceededTimeWindowOriginTimes, label="Exceeded Time Window", color="red", s=5)
#     #Add a Legend
#     ax.legend()
#     ax.set_xlabel('Request')
#     ax.set_ylabel('Time of Day')
#     ax.set_title('Time of completation of each request')
#     plt.savefig(path+"OriginTimeWindow.png")
#     plt.show()
    
#     #Create a line chart of the graphs
#     fig, ax = plt.subplots()
#     ax.plot(destinationTimeWindowList, label="Destination Time Window", color="black", alpha=0.5)
#     ax.scatter(completedInTimeDestinationKeys,completedInTimeDestinationTimes, label="Completed Destination Times", color="green", s=5)
#     ax.scatter(exceededTimeWindowDestinationKeys,exceededTimeWindowDestinationTimes, label="Exceeded Time Window", color="red", s=5)
#     ax.legend()
#     ax.set_xlabel('Request')
#     ax.set_ylabel('Time of Day')
#     ax.set_title('Time of completation of each request')
#     plt.savefig(path+"DestinationTimeWindow.png")
#     plt.show()
# def plotVehicleStats(vehilcesList,requestsList,path):
#     # #Complete a bar chart of each of the 10 vehicle's stats 
#     fig, ax = plt.subplots()
#     barWidth = 0.25
#     bars1 = [veh.stats._totalDistance/1000 for veh in vehilcesList]
#     bars2 = [veh.stats._totalPassengers for veh in vehilcesList]
#     bars3 = [veh.stats._totalRequests for veh in vehilcesList]
#     bars4 = [veh.stats._totalIdleTime/60 for veh in vehilcesList]

#     #Display bars1-4
#     r1 = np.arange(len(bars1))
#     r2 = np.arange(len(bars2))
#     r3 = np.arange(len(bars3))
#     r4 = np.arange(len(bars4))

#     #Labels X-Axis and Y-Axis
#     ax.set_xlabel('Vehicles')
#     ax.set_ylabel('Distance (Km)')
#     ax.set_title('Distance Travelled by Each Vehicle')

#     #Make each bar have a vertical label
#     ax.set_xticks([r for r in range(len(bars1))])
#     ax.set_xticklabels([f"{i+1}" for i in range(len(bars1))])
#     ax.bar(r1, bars1, color='black', width=barWidth, edgecolor='grey', label='Distance')

#     #Add text at top of column showing amount
#     for i in range(len(bars1)):
#         ax.text(i, bars1[i], str(round(bars1[i])), ha='center', va='bottom')
#     plt.savefig(path+"DistanceTravelled.png")
#     plt.show()

#     fig, ax = plt.subplots()
#     #Labels X-Axis and Y-Axis
#     ax.set_xlabel('Vehicles')
#     ax.set_ylabel('Total Passengers')
#     ax.set_title('Passengers Completed by Each Vehicle')

#     #Make each bar have a vertical label
#     ax.set_xticks([r for r in range(len(bars2))])
#     ax.set_xticklabels([f"{i+1}" for i in range(len(bars2))])
#     ax.bar(r2, bars2, color='black', width=barWidth, edgecolor='grey', label='Distance')
#     for i in range(len(bars2)):
#         ax.text(i, bars2[i], str(round(bars2[i])), ha='center', va='bottom')
#     plt.savefig(path+"PassengersCompleted.png")
#     plt.show()
    

#     fig, ax = plt.subplots()
#     #Labels X-Axis and Y-Axis
#     ax.set_xlabel('Vehicles')
#     ax.set_ylabel('Total Requests')
#     ax.set_title('Requests Completed by Each Vehicle')
#     #Make each bar have a vertical label
#     ax.set_xticks([r for r in range(len(bars3))])
#     ax.set_xticklabels([f"{i+1}" for i in range(len(bars3))])
#     ax.bar(r3, bars3, color='black', width=barWidth, edgecolor='grey', label='Distance')
#     for i in range(len(bars3)):
#         ax.text(i, bars3[i], str(round(bars3[i])), ha='center', va='bottom')
#     plt.savefig(path+"RequestsCompleted.png")
#     plt.show()
    
#     fig, ax = plt.subplots()
#     #Labels X-Axis and Y-Axis
#     ax.set_xlabel('Vehicles')
#     ax.set_ylabel('Total Idle Time (Minutes)')
#     ax.set_title('Idle Time of Each Vehicle')
#     #Make each bar have a vertical label
#     ax.set_xticks([r for r in range(len(bars4))])
#     ax.set_xticklabels([f"{i+1}" for i in range(len(bars4))])
#     ax.bar(r4, bars4, color='black', width=barWidth, edgecolor='grey', label='Distance')
#     for i in range(len(bars4)):
#         ax.text(i, bars4[i], str(round(bars4[i])), ha='center', va='bottom')
#     plt.savefig(path+"IdleTime.png")
#     plt.show()

In [1]:
# vehilceList = d1.vehicles
# requestList = d1.requests
# path = "./"
# plotTimeWindows(vehilceList,requestList,path)
# plotVehicleStats(vehilceList,requestList,path)